# Table of Contents
 <p>

READ (described in Sec. 3.2),
WRITE (described in Sec. 3.2).

ZERO(a, b) = 0, 
ONE(a, b) = 1, 
TWO(a, b) = 2, 

INC(a, b) = (a+1) mod M, 
ADD(a, b) = (a+b) mod M, 
SUB(a, b) = (a−b) mod M, 
DEC(a, b) = (a−1) mod M, 
LESS-THAN(a, b) = [a < b], 
LESS-OR-EQUAL-THAN(a, b) = [a ≤ b], 
EQUALITY-TEST(a, b) = [a = b], 
MIN(a, b) = mod M, 
SUB(a, b) = (a−b) mod M, 
DEC(a, b) = (a−1) mod M, 
LESS-THAN(a, b) = [a < b], 
LESS-OR-EQUAL-THAN(a, b) = [a ≤ b], 
EQUALITY-TEST(a, b) = [a = b], 
MIN(a, b) = min(a, b), 
MAX(a, b) = max(a, b), 
LESS-OR-EQUAL-THAN(a, b) = [a ≤ b], 
EQUALITY-TEST(a, b) = [a = b], 
MIN(a, b) = min(a, b), 
MAX(a, b) = max(a, b), 


In [1]:
import numpy as np
import theano

from theano import tensor
from collections import namedtuple
from theano.tensor.extra_ops import to_one_hot
from theano.tensor import roll
from theano.tensor import stack
from theano.tensor import batched_dot
from theano.tensor import concatenate
from theano.tensor import as_tensor
from theano.tensor import set_subtensor
from theano import shared
from numpy.random import uniform
from theano.tensor.nnet import softmax, relu, sigmoid
from theano import function, config, shared, sandbox, Out

def zero() :
    return to_one_hot(np.asarray([0]),M)
def one() :
    return to_one_hot(np.asarray([1]),M)
def two() :
    return to_one_hot(np.asarray([2]),M)
def get_const(value) :
    return to_one_hot(np.asarray([value % M]),M)
def create_memory_tape(init_val=0):
    m = stack([get_const(init_val) for i in range(M)], axis=1)
    #return shared(np.asarray(m.eval()), config.floatX) 
    return m
def inc(a) :
    return roll(a, 1, axis=1)
def negate(a) :
    return roll(a[:, ::-1], 1, axis=1)
def add(a,b) :
    rows = [roll(b[:,], j, axis=1) for j in range(M)]
    return (batched_dot(a, stack(rows, axis=1)))
def sub(a,b) :
    b_negative = negate(b)
    return add(a, b_negative)
def eq_zero(a) :
    r = tensor.zeros_like(as_tensor(a))
    r = set_subtensor(r[:,1], a[:, 0])
    r = set_subtensor(r[:, 0], 1 - a[:, 0])
    return r
def lt(a,b):
    a = as_tensor(a)
    b = as_tensor(b)
    b = set_subtensor(b[:,0], [0]) 
    b = roll(b[:,], -1, axis=1)
    rows = [set_subtensor(tensor.zeros_like(as_tensor(b))[:,j:M], b[:,j:M]) for j in range(M)]
    result = ((tensor.dot(as_tensor(a), stack(rows, axis =1))).sum()).dimshuffle('x')
    return set_subtensor(tensor.zeros_like(as_tensor(b))[:,1], result)
def eq(a,b):
    elemwise_result = tensor.eq(as_tensor(a),as_tensor(b))
    sum = elemwise_result.sum()
    result = tensor.eq(sum,M).dimshuffle('x')
    return set_subtensor(tensor.zeros_like(as_tensor(a))[:,1], result)
def max_(a,b):
    if(tensor.eq([0],lt(a,b))):
        return as_tensor(b)
    else:
        return as_tensor(a)
def min_(a,b):
    if(tensor.eq([1],lt(a,b))):
        return as_tensor(a)
    else:
        return as_tensor(b)    
def read(mem, a) :
    ptr = as_tensor(a)
    return weighted_avg(mem, ptr), mem
def write(mem, a, b) :
    ptr = as_tensor(a)
    val = as_tensor(b)
    J = (tensor.ones_like(zero())).T
    erase_vector = (J-ptr.T).dot(J.T)
    x = tensor.mul(erase_vector, mem)
    y = (ptr.T).dot(val)
    mem = (x + y)
    return val, mem
def write_external(a, b, ext_mem) :
    ptr = as_tensor(a)
    val = as_tensor(b)
    J = (tensor.ones_like(zero())).T
    erase_vector = (J-ptr.T).dot(J.T)
    x = tensor.mul(erase_vector, ext_mem)
    y = (ptr.T).dot(val)
    ext_mem = (x + y)
    return ext_mem

def get_registers(init_val):
    return stack([get_const(init_val) for i in range(R)], axis=1)

def weighted_avg(inputs, coefficient) :
    return batched_dot(inputs.transpose(0, 2, 1), coefficient.dimshuffle(0, 1, 'x')).flatten(2);

def compute_gate_new(module, inputs, coefficients, memory_tape, debug=False) :
    """ Arity of this gate must be equal to the number of given
        coefficients list
    """ 
    if (len(coefficients) != module.arity) :
        print("Error: Incorrect number of coefficients: ",  len(coefficients), " to module arity: ", module.arity)
    
    params = [weighted_avg(inputs, as_tensor(coef)) for coef in coefficients]
    
    if (debug == True):
        for i, p in enumerate(params):
            print("compute_gate_new: coeff", coefficients[i].eval(), " weighted param [", i ,"] = ", p.eval().argmax())
    
    if (module.memory_function == True) :
        #print("read/write ")
        output, memory_tape = module.function(memory_tape, *params)
    else :   
        output = module.function(*params)
    
    #error check for constant gates
    return output, memory_tape


def get_n_tensor(t, count, idx):
    result=[]
    if count > 0: 
        result = [t[idx+i] for i in range(count)]
    return result, (idx+count)

def fuzzy_circuit(registers, gates, memory_tape, gate_coef, reg_coef, debug=False):
    # Initially, only the registers may be used as inputs.
    Q= len(gates)
    gate_inputs = registers
    idx=0    
    
    # Run through all the gates.
    for i in range(Q):
        c, idx = get_n_tensor(gate_coef, gates[i].arity, idx) 
        if (debug==True):
            print("gate i = ", i, "arity:", gates[i].arity, "gate inputs: ", gate_inputs.eval().argmax(axis=2))
        output, memory_tape = compute_gate_new(gates[i], gate_inputs, c, memory_tape, debug)
        # Append the output of the gate as an input for future gates.
        gate_inputs = concatenate([gate_inputs, output.dimshuffle(0,'x',1)], axis=1)
        if (debug==True):
            print("gate output: ", output.eval().argmax())    
            print("concatenated inputs: ", gate_inputs.eval().argmax(axis=2))       
        
    # All leftover coefficients are for registers.
    new_registers = []
             
    for i in range(len(reg_coef)):
        # (R+Q) x M dot 1 X (R+Q) 
        new_registers.append(weighted_avg(gate_inputs, reg_coef[i]))
        if (debug == True) :
            print("register [", i, "]  new value: reg_coef", reg_coef[i].eval().argmax(),"weighted_avg of gate inputs and reg_coef: ", weighted_avg(gate_inputs, reg_coef[i]).eval().argmax())
    return tensor.stack(new_registers, axis=1), memory_tape


# coefficients = [r1,r2..rR,g1_param1,g1_param2,...,gQ_param1,gQ_param2,c1,c1..CR,cR+1,..cR+Q]
def gen_random_weights(layer1, layer2, dtype=np.float64, _min=-1, _max=1):
    weights = uniform(low=_min, high=_max, size=(layer1, layer2))
    var = shared(weights.astype(dtype), name="w{0}x{1}".format(layer1, layer2))   
    #var = tensor.addbroadcast(var, 0)
    return var

def gen_network_weights(gates, layers):
    n_registers = R #input is R registers
    w = []
    current_layer_units = n_registers
    for next_layer_units in layers:
        w.append(gen_random_weights(current_layer_units + 1, next_layer_units))
        current_layer_units = next_layer_units
    #output wt for gate coefficients
    gate_coef = []
    for i, gate in enumerate(gates):
        print(i, gate)
        #each param to gates are R+i depending on arity 
        gate_output_units = n_registers + i
        for _ in range(gate.arity):
            gate_coef.append(gen_random_weights(current_layer_units + 1, gate_output_units))            
    
    #print("gen_network_weights => gate_coef", gate_coef)         
    #output wt for new registers
    reg_coef = []
    reg_output_units = n_registers + len(gates)
    for _ in range(n_registers):
        reg_coef.append(gen_random_weights(current_layer_units + 1, n_registers + len(gates)))    
    
    #print("gen_network_weights => reg_coef", reg_coef)      
    prob_completion_coef = (gen_random_weights(current_layer_units + 1, 1))

    for i in gate_coef:
        w.append(i)

    for x in reg_coef:
        w.append(x)
    
    w.append(prob_completion_coef)   
    return w

def aug_ones_col(inputs):
    print("OMFG")
    s = inputs.shape
    col_elems = s[1]
    ones = tensor.ones_like(to_one_hot(np.asarray([0]), col_elems))
    x = concatenate([inputs.T,ones], axis=1)
    return x.T

def aug_ones_col_new(inputs):
    x=[]
    s = inputs.shape
    d = inputs.ndim
    row_elems = s[d-2]
    matrix = tensor.reshape(inputs, (s[d-2],s[d-1]))
    matrix = tensor.shape_padleft(matrix)
    ones = tensor.ones_like(to_one_hot(np.asarray([0]), row_elems))
    x = concatenate([matrix.dimshuffle(0,2,1),ones.dimshuffle(0,'x',1)], axis=1)
    return x.dimshuffle(0,2,1)

def controller_forward_prop(n_registers, layers, weight_matrix, gates, registers, debug) :
    inputs = aug_ones_col_new(registers[:,:,0])
    if (debug == True):
        #print("registers: ", registers.eval())
        #print("registers[:,:,0]: ", registers[:,:,0].eval())
        print("inputs: ", inputs.eval())
        
    for i in range(len(layers)):
        W = weight_matrix[i]
        inputs = aug_ones_col_new(relu(inputs.dot(W)))  
        if (debug == True):
            print("compute layers inputs: ", inputs.eval())
    
    #extract gate coefficients
    gate_coef = []
    n_gate_coef = 0
    
    for i, gate in enumerate(gates): 
        #each param to gates are R+i depending on arity 
        gate_output_units = n_registers + i
        for _ in range(gate.arity):
            n_gate_coef += 1  
            
    for W in weight_matrix[len(layers):len(layers) + n_gate_coef]:
        gate_coef.append(softmax(inputs.dot(W)[0]))            
    
    reg_coef = []
    for W in (weight_matrix[len(layers)+n_gate_coef:len(layers)+n_gate_coef+n_registers]):
        reg_coef.append(softmax(inputs.dot(W)[0]))  
    
    if (debug == True):
        #print("inputs.dot(W)", inputs.dot(W).eval())
        #print("inputs.dot(W)[0]", inputs.dot(W)[0].eval())
        #print("p_complete inputs: ", inputs.eval())
        print("inputs.dot(weight_matrix[-1]: ", inputs.dot(weight_matrix[-1]).eval())  
              
    p = sigmoid((inputs.dot(weight_matrix[-1])))
    if (debug == True):
        print("p_complete: ", p.eval())
        
    return p, reg_coef, gate_coef

def calculate_cost_at_t(prob_complete_t, t, cum_error, cum_cost, cum_prob_t, p_incomplete, memory_in, desired_output, output_len, debug) :
    e_min = 1e-100
    e_max = 1e+100
    cost_t = 0
    entropy = 0
    #TODO: Find use for desired registers in calculating cost. Now, only desired memory layout is matched.
    for i in range(output_len):
        #Compute the loss for this register using the mask.
        y = to_one_hot(desired_output.argmax(axis=2)[:,i], M)   
        y_hat = to_one_hot(memory_in.argmax(axis=2)[:,i], M) 
        ln_y_hat = tensor.log(tensor.clip(memory_in[:, i, :], e_min, e_max))
        ln_1_minus_y_hat = tensor.log(tensor.clip(1 - memory_in[:, i, :], e_min, e_max))
        x1 = y * ln_y_hat
        x2 = (1 - y) * ln_1_minus_y_hat
        loss = (x1+x2).sum(axis=1) 
        cost_t += tensor.shape_padright(loss, 1)
        entropy += ((memory_in[:, i, :]) * ln_y_hat).sum(axis=1)
        if (debug == True):
            #print("desired_output at i", desired_output[:,i].eval())
            #print("memory value at i", memory_in[:, i, :].eval())
            #print("y ", y.eval())
            #print("y_hat ", y_hat.eval())
            #print("ln_y_hat ", ln_y_hat.eval())
            #print("ln_1_minus_y_hat ", ln_1_minus_y_hat.eval())
            #print("x1 ", x1.eval())
            #print("x2 ", x2.eval())
            print("loss ", loss.eval())
            print("cost_t ", cost_t.eval())
            print("entropy ", entropy.eval())
            #print("tensor.shape_padright(loss, 1) ", tensor.shape_padright(loss, 1).eval())
            #print("Desired value at mem location [", i, "] is ", y.eval().argmax(), "but actual value is [", memory_in[:,i,:].eval().argmax(), "loss is ", loss.eval(), " cost = ", cost_t.eval()) 
    
    if (t == MAX_TIMESTEP):
        prob_complete = 1 - cum_prob_t
        #penalize cost if execution not over (cost_t should be zero)
        cum_cost = cum_cost - cum_error*cost_t
    else:
        prob_complete = (prob_complete_t * p_incomplete)
    
    cum_prob_t += prob_complete
    p_incomplete *= (1 - prob_complete_t)
    cum_cost -= (cost_t*prob_complete)
    cum_cost -= entropy * (1/(1 + t)**0.55)
    cum_error = cum_error - cost_t
    
    if (debug == True):
        print("prob_complete_t", prob_complete_t.eval())
        print("p_incomplete", p_incomplete.eval())
        print("prob_complete", prob_complete.eval())
        print("cum_prob_t", cum_prob_t.eval())
        print("cost_t*prob_complete", (cost_t*prob_complete).eval())
        print("cum_cost", cum_cost.eval())
        print("cum_error", cum_error.eval())
    return cum_error, cum_cost, cum_prob_t, p_incomplete

def machine_compute_step_t(debug, R, layers, w, gates, t, desired_output, output_len, registers, memory_tape, cum_error, cost_t, cum_prob, prob_incomplete) : 
    prob_complete_t, reg_coef, gate_coef = controller_forward_prop(R, layers, w, gates, registers, debug)
    new_registers, new_memory_tape = fuzzy_circuit(registers, gates, memory_tape, gate_coef, reg_coef, debug)
    cum_error, cost_t, cum_prob_t, prob_incomplete = calculate_cost_at_t(prob_complete_t, t, cum_error, cost_t,  cum_prob, prob_incomplete, new_memory_tape, desired_output, output_len, debug)
    return new_registers, new_memory_tape, cum_error, cost_t, cum_prob_t, prob_incomplete

def compute_all_timesteps(gates, layers, registers, memory_tape, w, reg_lambda, output_len):
    #w = make_broadcastable(w)
    # Create symbolic variables for the input to the machine
    # and for the desired output of the machine.
      
    #initial_registers = registers
    #desired_output = memory_tape

    initial_registers = tensor.dtensor3("Registers")
    initial_memory = tensor.dtensor3("Memory_Tape")
    desired_output = tensor.dtensor3("Y")
     
    
    # Run the model for all timesteps. The arguments are 
    # registers, cost, cumulative probability complete, 
    # and probability incomplete. The latter are initialized
    # to zero and to one, respectively.
    
    v0 = as_tensor(0)
    v1 = as_tensor(1)
    output = [initial_registers, initial_memory, v0, v0, v0, v1]
    intermediate_registers = []
    for timestep in range(MAX_TIMESTEP):
        print("compute_all_timesteps t = ", timestep)
        output = machine_compute_step_t(False, R, layers, w, gates, timestep+1, desired_output, output_len, *(output))
        intermediate_registers.append(output[0])

    # Add in regularization, to avoid overfitting simple examples.
    reg_cost = reg_lambda * sum((p * p).sum() for p in list(w))
    
    # Get the final cost: regularization plus loss.
    final_cost = reg_cost + output[3].sum()
    
    # Return the symbolic variables, the final cost, and the
    # intermediate register values for analysis and prediction.
    return initial_registers, initial_memory, desired_output, final_cost, intermediate_registers


In [34]:
#Maximum Representable Integer M is set below
M = 16
# Number of registers
R = 5
#Max number of timesteps
MAX_TIMESTEP = 15

Module = namedtuple("Module", "arity function memory_function")

m_zero = Module(0, zero, False)
m_one = Module(0, one, False)
m_two = Module(0, two, False)
m_inc = Module(1, inc, False)
m_negate = Module(1, negate, False)
m_add = Module(2, add, False)
m_sub = Module(2, sub, False)
m_eq_zero = Module(1, eq_zero, False)
m_read = Module(1, read, True)
m_write = Module(2, write, True)
m_lt = Module(2, lt, False)
m_eq = Module(2, eq, False)
m_min = Module(2, min_, False)
m_max = Module(2, max_, False)

gates = [m_read, m_inc, m_add, m_min, m_write]
#gates = [m_read, m_write]
N = len(gates)

registers = get_registers(1)
memory_tape = create_memory_tape()
desired_out = create_memory_tape(0)
for i in range(M):
     desired_out = write_external(get_const(i), get_const(i), desired_out)
        
layers = [5,5]
w = gen_network_weights(gates, layers)
print("weight matrix: ", w)

(0, Module(arity=1, function=<function read at 0x106cc5758>, memory_function=True))
(1, Module(arity=1, function=<function inc at 0x106cc5320>, memory_function=False))
(2, Module(arity=2, function=<function add at 0x106cc5410>, memory_function=False))
(3, Module(arity=2, function=<function min_ at 0x106cc56e0>, memory_function=False))
(4, Module(arity=2, function=<function write at 0x106cc57d0>, memory_function=True))
('weight matrix: ', [w6x5, w6x5, w6x5, w6x6, w6x7, w6x7, w6x8, w6x8, w6x9, w6x9, w6x10, w6x10, w6x10, w6x10, w6x10, w6x1])


In [17]:
reg_lambda = 0.1
output_len = 5
result  = compute_all_timesteps(gates, layers, registers, memory_tape, w, reg_lambda, output_len)
initial_registers, initial_memory, desired_output, final_cost, intermediate_registers = result

('compute_all_timesteps t = ', 0)
('compute_all_timesteps t = ', 1)
('compute_all_timesteps t = ', 2)
('compute_all_timesteps t = ', 3)
('compute_all_timesteps t = ', 4)
('compute_all_timesteps t = ', 5)
('compute_all_timesteps t = ', 6)
('compute_all_timesteps t = ', 7)
('compute_all_timesteps t = ', 8)
('compute_all_timesteps t = ', 9)


In [18]:
initial_registers, initial_memory, desired_output, final_cost, intermediate_registers

(Registers,
 Memory_Tape,
 Y,
 Elemwise{add,no_inplace}.0,
 [Join.0,
  Join.0,
  Join.0,
  Join.0,
  Join.0,
  Join.0,
  Join.0,
  Join.0,
  Join.0,
  Join.0])

In [19]:
gradients = theano.grad(final_cost, list(w)) #, disconnected_inputs='warn', return_disconnected='Disconnected')


In [20]:
for i in range(len(gradients)):
    theano.gradient.grad_clip(gradients[i], -1, 1)

In [21]:
import time
t0 = time.time()  
# Compile training function to compute gradients.
train = theano.function([initial_registers, initial_memory, desired_output], [final_cost] + gradients) #, on_unused_input='ignore', allow_input_downcast=True)
t1 = time.time()  
print ("Training took", t1 - t0, "seconds")  

('Training took', 471.1658170223236, 'seconds')


In [22]:
#Access Task
input = [ 7,  1,  12,  4,  7,  12,  1,  13,  8,  2, 1, 3, 11, 11, 12, 0]
output = [ 13 ]
registers = get_registers(0)
memory_tape = stack([get_const(i) for i in input], axis=1)
desired_out = stack([get_const(i) for i in output], axis=1)

In [23]:
#Permutation Task
input = [ 6,  1,  3,  2,  5,  4,  9,  13,  8,  2, 1, 0, 0, 0, 0, 0]
output = [ 9, 8, 13, 1, 2]
registers = get_registers(0)
memory_tape = stack([get_const(i) for i in input], axis=1)
desired_out = stack([get_const(i) for i in output], axis=1)

In [24]:
print ("desired_out.eval()", desired_out.eval().argmax(axis=2))
print ("memory_tape.eval()", memory_tape.eval().argmax(axis=2))
print ("registers.eval()", registers.eval().argmax(axis=2))

('desired_out.eval()', array([[ 9,  8, 13,  1,  2]]))
('memory_tape.eval()', array([[ 6,  1,  3,  2,  5,  4,  9, 13,  8,  2,  1,  0,  0,  0,  0,  0]]))
('registers.eval()', array([[0, 0, 0, 0, 0]]))


In [25]:
for i in range(1):
    output_train = train(registers.eval(), memory_tape.eval(), desired_out.eval())
    cost = output_train[0]
    gradients = output_train[1:]
    print(output_train)

[array(32566.870061579906), array([[  1.56772140e-02,   4.68593674e-03,   8.11880607e-02,
          2.34807082e+02,   6.62956322e-03],
       [  1.89930322e-01,  -1.21110046e-01,  -1.10753222e-01,
          2.35082040e+02,   5.00692735e-02],
       [ -1.99829372e-01,  -5.11300890e-02,  -1.61575687e-01,
          2.35018937e+02,  -1.73288272e-01],
       [ -1.50021053e-01,   3.83501191e-02,  -7.81398272e-02,
          2.35052690e+02,   8.07285554e-02],
       [  8.08193604e-02,   1.88556788e-01,   1.36148384e-01,
          2.34942764e+02,  -1.58006263e-01],
       [ -1.72135284e-01,  -1.07849495e-01,  -1.42119016e-01,
          2.34929385e+02,  -1.92288685e-02]]), array([[  6.94147563e-03,  -1.48663567e-01,   1.73079969e-01,
         -1.00802647e-01,   6.87998768e-02],
       [ -1.62503148e-01,   1.46361431e-01,  -1.48898013e-01,
         -1.46055339e-01,   3.76768882e-02],
       [  1.31528884e-01,   1.28943219e-01,   3.62986464e-03,
          3.00508273e-02,   1.25315576e-01],
       

In [26]:
#training data for "access" 
train_data_size = 50
train_inputs_access = []
train_outputs_access = []

input = []
output = []
for i in range(train_data_size):
    #generated a random array
    input = np.random.randint(1,15,size=(M))
    input_t = stack([get_const(i) for i in input], axis=1)
    #print("input array", input_t.eval().argmax(axis=2))
    train_inputs_access.append(input_t)
    first_elem = input[0]
    output = np.array([[input[first_elem]]])
    output_t = stack([get_const(i) for i in output], axis=1)
    #print(output_t.eval().argmax(axis=2))
    train_outputs_access.append(output_t)
print(train_inputs_access)    
print(train_outputs_access)    


[Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0]
[Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0]


In [27]:
#training data for "permutation" 
train_data_size = 50
train_inputs_access = []
train_outputs_access = []

start_idx = 6
start = [start_idx]
end = [0]
array_len = 5
#array_len = np.random.randint(5,M-start_idx-1)
for i in range(train_data_size):
    P = np.arange(array_len)
    np.random.shuffle(P)
    A = np.random.randint(1, M, size=M-6)
    A[array_len] = 0
    input = np.concatenate((start, P, A), axis=0)
    input_t = stack([get_const(i) for i in input], axis=1)
    #print("input array", input_t.eval().argmax(axis=2))
    train_inputs_access.append(input_t)
    output = np.array([A[idx] for idx in P])
    output_t = stack([get_const(i) for i in output], axis=1)
    #print("output array", output_t.eval().argmax(axis=2))
    train_outputs_access.append(output_t)
print(train_inputs_access)    
print(train_outputs_access)    


[Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0]
[Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0, Join.0]


In [41]:
def clamp(n, minn, maxn):
    for i, item in enumerate(n):
        n[i] = (map(lambda x: max(min(maxn, x), minn), item))
    return n    
    
def get_l2_norm(w):
    x=0
    for i, item in enumerate(w):
        x += np.linalg.norm(item)
    return x    

def add_noise(gradients, mean, std_dev):
    result = map(lambda x: x + np.random.normal(mean,std_dev,x.shape), gradients)
    return result


def adam_optimize(params, train, registers, train_inputs, train_outputs, output_len,
                  alpha=0.001, b1=0.9, b2=0.999,
                  epsilon=1e-8, batch_size=1):
    """Implementation of Adam optimization method, with hyperparameters
    taken as recommended by the original publication."""
    # Initialize first and second moment estimates to zero.
    # This causes some bias, which is addressed later.
    moment1 =  [0 for _ in params]
    moment2 = [0 for _ in params]
    
    timestep = 0  # Current optimization step
    batch = 0     # Where does this batch start
    
    converged = False
    gradients = []
    
    while not converged:
        timestep += 1
        
        inputs  = train_inputs[batch:batch+batch_size][0]
        outputs = train_outputs[batch:batch+batch_size][0] 
        
        #print(inputs.eval())
        #print(outputs.eval())
        
        result = train(registers, inputs.eval(), outputs.eval())
        
        cost = result[0]
        gradients = result[1:]
        
        gradients=add_noise(gradients, 0, 1/(1 + timestep)**0.55)
        
        # Advance to next batch.
        batch = (batch + batch_size) % len(train_inputs)

        # Compute first and second moment estimates.
        # These are decaying moving averages; first moment
        # uses the gradient, second uses squared gradient.
        moment1  = [b1 * m + (1 - b1) * gradient
                    for (m, gradient)
                    in zip(moment1, gradients)]
        moment2 = [b2 * v + (1 - b2) * gradient ** 2
                   for (v, gradient)
                   in zip(moment2, gradients)]
        
        # Correct for initialization bias and compute new values.
        correction1 = 1. / (1 - b1 ** timestep)
        correction2 = 1. / (1 - b2 ** timestep)
        corrected1 = [correction1 * m for m in moment1]
        corrected2 = [correction2 * v for v in moment2]
        
        # Compute new parameter values.
        params_new = [p.get_value() - alpha * m1 / (np.sqrt(m2) + epsilon)
                      for (p, m1, m2) in zip(params, corrected1, corrected2)]

        # Check for convergence by looking at magnitude of delta.
        delta = [abs(p.get_value() - p_new)
                 for (p, p_new) in zip(params, params_new)]
        converged = all((d < 0.5 * alpha).all() for d in delta)        
        
        # Update parameters to new values.
        for p, p_new in zip(params, params_new):
            p_new = clamp(p_new, -1, 5)
            p.set_value(p_new.astype('float32'))
            
        # Provide some output for tracking during runtime.
        if timestep % 100 == 1 or converged:
            print("Cost (t = %4d): \t%.2f" % (timestep - 1, cost))
            print("l2 norm of gradients: ", get_l2_norm(gradients))
            print("l2 norm of p_new: ", get_l2_norm(p_new))

In [42]:
#def adam_optimize(params, train, train_inputs, mem, train_outputs, output_len,
#                  alpha=0.001, b1=0.9, b2=0.999,
#                  epsilon=1e-8, batch_size=1):
    
result = adam_optimize(w, train, registers.eval(), train_inputs_access, train_outputs_access, output_len)

Cost (t =    0): 	105640.50
('l2 norm of gradients: ', 281789.40697129804)
('l2 norm of p_new: ', 5.9948665952700306)
Cost (t =  100): 	105640.50
('l2 norm of gradients: ', 281789.40697129804)
('l2 norm of p_new: ', 6.5598948680308338)
Cost (t =  200): 	105640.50
('l2 norm of gradients: ', 281789.40697129804)
('l2 norm of p_new: ', 7.1243232976148976)
Cost (t =  300): 	105640.50
('l2 norm of gradients: ', 281789.40697129804)
('l2 norm of p_new: ', 7.6884925376849358)
Cost (t =  400): 	105640.50
('l2 norm of gradients: ', 281789.40697129804)
('l2 norm of p_new: ', 8.2525565404827663)
Cost (t =  500): 	105640.50
('l2 norm of gradients: ', 281789.40697129804)
('l2 norm of p_new: ', 8.8165649779983486)


KeyboardInterrupt: 

In [43]:
for W in w:
    print(W.eval())

[[-1.5820272  -1.58195782 -1.39006913 -1.46808684 -1.5820272 ]
 [-1.5820272   1.85517943 -1.36681986 -1.17259645 -1.5820272 ]
 [-1.5820272   1.9800458  -0.67167199 -1.4658426   1.9800458 ]
 [ 1.9800458  -0.82227963 -0.48235503 -1.45985949  1.9800458 ]
 [-1.5820272   1.97981215 -1.3075155  -1.46052396 -1.5820272 ]
 [ 1.9800458   1.75512171  0.08379318 -1.54254603  1.9800458 ]]
[[ 1.9800458   1.9800458  -1.5820272   1.9800458   1.9800458 ]
 [-1.5820272  -1.58195782  0.7977922   1.9800458  -1.58195782]
 [-1.58181906 -1.53766978  1.86326826 -1.53687298 -1.53352511]
 [ 0.83942389 -1.53775883  1.50290775 -1.53697324 -1.53362954]
 [ 1.05409169  1.9800458  -1.5820272   1.9800458   1.9800458 ]
 [ 1.14313841 -1.53770864  1.86337221 -1.53691614 -1.53356898]]
[[-1.58195782  1.97981215 -1.5820272   1.97992897  0.91708648]
 [-0.87685418  0.56759787 -1.08618116  0.25055104 -0.78728998]
 [-0.12010951  0.35268214  0.23217726  0.87799788  1.1698575 ]
 [-0.15461349 -0.90247059 -1.09806836 -0.43884385 -0.

In [44]:
#Access Task
input = [ 7,  1,  12,  4,  7,  12,  1,  13,  8,  2, 1, 3, 11, 11, 12, 0]
output = [ 13 ]
registers = get_registers(0)
memory_tape = stack([get_const(i) for i in input], axis=1)
desired_out = stack([get_const(i) for i in output], axis=1)

In [45]:
#Permutation Task
input = [ 6,  1,  3,  2,  5,  4,  9,  13,  8,  2, 1, 0, 0, 0, 0, 0]
output = [ 9, 8, 13, 1, 2]
registers = get_registers(0)
memory_tape = stack([get_const(i) for i in input], axis=1)
desired_out = stack([get_const(i) for i in output], axis=1)
output_len = 5

In [46]:
v0 = as_tensor(0)
v1 = as_tensor(1)
output = [registers, memory_tape, v0, v0, v0, v1]
new_registers = []
print("registers = ", registers.eval().argmax(axis=2))
print("memory tape:  ", memory_tape.eval().argmax(axis=2))
print("desired tape: ", desired_out.eval().argmax(axis=2))
for timestep in range(MAX_TIMESTEP):
    output = machine_compute_step_t(False, R, layers, w, gates, timestep+1, desired_out, output_len, *(output))
    registers, memory_tape, cum_error, cost, cum_prob_t, prob_incomplete = output    
    print("registers = ", registers.eval().argmax(axis=2))
    print("memory tape:  ", memory_tape.eval().argmax(axis=2))
    print("desired tape: ", desired_out.eval().argmax(axis=2))
    print("cum cost = ", cost.eval())
    print("cum error = ", cum_error.eval())
    print("prob_incomplete = ", prob_incomplete.eval())
    

('registers = ', array([[0, 0, 0, 0, 0]]))
('memory tape:  ', array([[ 6,  1,  3,  2,  5,  4,  9, 13,  8,  2,  1,  0,  0,  0,  0,  0]]))
('desired tape: ', array([[ 9,  8, 13,  1,  2]]))
('registers = ', array([[6, 0, 0, 1, 0]]))
('memory tape:  ', array([[ 0,  1,  3,  2,  5,  4,  9, 13,  8,  2,  1,  0,  0,  0,  0,  0]]))
('desired tape: ', array([[ 9,  8, 13,  1,  2]]))
('cum cost = ', array([[[ 1.12074194]]]))
('cum error = ', array([[ 1186.88086259]]))
('prob_incomplete = ', array([[[ 1.]]]))
('registers = ', array([[ 9,  1,  0,  1, 12]]))
('memory tape:  ', array([[ 0,  1,  3,  2,  5,  4,  9, 13,  8,  2,  1,  0,  0,  0,  0,  0]]))
('desired tape: ', array([[ 9,  8, 13,  1,  2]]))
('cum cost = ', array([[[ 2.21756133]]]))
('cum error = ', array([[ 1250.66919713]]))
('prob_incomplete = ', array([[[ 1.]]]))
('registers = ', array([[1, 1, 1, 2, 9]]))
('memory tape:  ', array([[ 0,  1,  3,  2,  5,  4,  9, 13,  8,  2,  1,  0,  0,  0,  0,  0]]))
('desired tape: ', array([[ 9,  8, 13,  1, 

KeyboardInterrupt: 